## <b> Tools (Inbuilt) </b> 

In [6]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [7]:
## use the inbuilt tool of wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki) ## gives the tool
wiki.name

'wikipedia'

In [ ]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv) ## gives the tool
arxiv.name

'arxiv'

In [9]:
tools = [wiki, arxiv]

## <b> Custom Tool  (RAG Tool) </b>

In [15]:
from dotenv import load_dotenv

load_dotenv()

import os, sys

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [16]:
loader = WebBaseLoader("https://python.langchain.com/docs/introduction/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)

vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000029C419A67B0>, search_kwargs={})

In [17]:
# now we can create this retriever as tool, for that we need create_retriever_tool.

from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever, "langchain-search", "Search any Information about langchain.")
retriever_tool.name

'langchain-search'

In [18]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\My Learning\\Langchain\\Projects\\3.Search-Engine-With-Tools-Agent\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langchain-search', description='Search any Information about langchain.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000029C3E7B8A40>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.fai

## <b> Agents </b>

In [20]:
import os, sys, openai
from dotenv import load_dotenv

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [32]:
## run alll this tools with the Agents, and LLM models

from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key= groq_api_key, model_name = "llama-3.1-8b-instant")   

In [33]:
## Prompt template

from langchain import hub ## used this hub for usage of specific type of prompt structures, it is another way to write prompt templates
## but always use chatpromptTemplate for createting prompts

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [34]:
from langchain.agents import create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [35]:
## Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent= agent, tools= tools, verbose= True)

In [36]:
agent_executor.invoke({"input":" Tell me about the langchain."})



> Entering new AgentExecutor chain...

Invoking: `langchain-search` with `{'query': 'langchain'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Introduction | 🦜️🔗 LangChain

Skip to main contentThese docs will be deprecated and no longer maintained with the release of LangChain v1.0 in October 2025. Visit the v1.0 alpha docsIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextH

{'input': ' Tell me about the langchain.',
 'output': "It appears that LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It provides a set of tools and libraries to build, deploy, and manage LLM-based applications. LangChain is designed to be flexible and adaptable, allowing developers to build applications that can be used in a variety of settings, from chatbots and virtual assistants to content generation and more.\n\nLangChain's architecture is based on a modular design, which allows developers to easily integrate different components and build custom applications. The framework provides a range of tools and libraries for tasks such as data processing, model training, and inference, as well as for building and deploying applications.\n\nLangChain is a relatively new framework, and it is still evolving. However, it has already gained a significant following among developers and researchers who are interested in 

In [37]:
agent_executor.invoke({"input":"what are the langchain components."})



> Entering new AgentExecutor chain...

Invoking: `langchain-search` with `{'query': 'langchain components'}`


LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Introduction | 🦜️🔗 LangChain

langchain-core: Base abstractions for chat models and other components.
Integration packages (e.g. langchain-openai, langchain-anthropic, etc.): Important integrations have been split into lightweight packages that are co-maintained by the LangChain team and the integration developers.
langchain: Chains, agents, and retrieval strategies that make up an application's cognitive architecture.
langchain-community: Third-party integrations that are community maintained.
langgraph: Orchestration framework for combining LangChain components into production-ready applications with persistence, streaming, and other key features. See LangGraph documentation.

Guides​
Tutorials​
If you're looking 

{'input': 'what are the langchain components.',
 'output': "The main components of LangChain include:\n\n1. langchain-core: Provides base abstractions for chat models and other components.\n2. Integration packages: Important integrations have been split into lightweight packages that are co-maintained by the LangChain team and the integration developers.\n3. langchain: Offers chains, agents, and retrieval strategies that make up an application's cognitive architecture.\n4. langchain-community: Includes third-party integrations that are community maintained.\n5. langgraph: An orchestration framework for combining LangChain components into production-ready applications with persistence, streaming, and other key features.\n\nThese components work together to simplify the LLM application lifecycle and enable the development of a wide range of applications."}

In [38]:
agent_executor.invoke({"input":" What is machine lerning? "})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks wit
Invoking: `langchain-search` with `{'query': 'machine learning'}`


Select chat model:Google Gemini▾OpenAIAnthropicAzureGoogle GeminiGoogle VertexAWSGroqCohereNVIDIAFireworks AIMistral AITogether AIIBM watsonxDatabricksxAIPerplexityDeepSeekChatOCIGenAIpip install -qU "langchain[google-genai]"import getpassimport osif not os.environ.get("GOOGLE_API_KEY"):  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")from langchain.chat_models import init_chat_modelmodel = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
model.invoke("Hello, world!")
noteThese docs focus on the Python LangChain library

{'input': ' What is machine lerning? ',
 'output': 'It looks like my previous response got cut off. Here is the complete response:\n\n <function=langchain-search>{"query": "machine learning"}</function>'}

In [40]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on c
Invoking: `wikipedia` with `{'query': 'Attention Is All You Need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `wikipedia` with `{'query': 'Transformer (machine learning)'}`


Page: Transformer (deep learning architecture)
Summary: In deep learning, transformer is a neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token 
Invoking: 

{'input': "What's the paper 1706.03762 about?",
 'output': 'Agent stopped due to max iterations.'}